In [1]:
import json
import pandas as pd
import string
import regex as re
import pickle

from pathlib import Path
from thoth.lab import common
from thoth.lab import solver
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
#from pyonmttok import Tokenizer
from nltk.corpus import stopwords
from string import punctuation

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/skotak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
FILE_NAME = "thoth-solver-dataset-v1.0.zip"
#common.extract_zip_file(FILE_NAME)

In [4]:
solver_reports = solver.aggregate_solver_results()
#solver_reports = solver.aggregate_solver_results(is_local=False)

In [5]:
solver_reports_extracted_data = []
solver_errors = []
for solver_document in solver_reports:
    solver_report_extracted_data = solver.extract_data_from_solver_metadata(solver_document["metadata"])
    for k, v in solver_document["result"].items():
        solver_report_extracted_data[k] = v
        if k == "errors" and v:
            errors = solver.extract_errors_from_solver_result(v)
            for error in errors:
                solver_errors.append(error)
    
    packages = solver.extract_tree_from_solver_result(solver_document["result"])
    solver_report_extracted_data["packages"] = packages
    solver_reports_extracted_data.append(solver_report_extracted_data)


In [6]:
solver_total_errors_df = pd.DataFrame(solver_errors)

solver_total_errors_df.head(10)

,package_name,package_version,index_url,type,command,message,return_code,stderr,stdout,timeout
0,tryton,3.2.20,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting tryton==3.2.20\n Downloading https...,60.0
1,pyobjc-framework-contacts,4.0b1,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting pyobjc-framework-contacts==4.0b1\n ...,60.0
2,numpy,1.14.0,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting numpy==1.14.0\n Downloading https:...,60.0
3,hachoir-core,1.3,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting hachoir-core==1.3\n Downloading ht...,60.0
4,xattr,0.7.7,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting xattr==0.7.7\n Downloading https:/...,60.0
5,spacy,2.1.1.dev0,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting spacy==2.1.1.dev0\n Downloading ht...,60.0
6,fastavro,0.16.5,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting fastavro==0.16.5\n Downloading htt...,60.0
7,pyeclib,1.6.0,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting pyeclib==1.6.0\n Downloading https...,60.0
8,happybase,0.8,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting happybase==0.8\n Downloading https...,60.0
9,mahotas,0.6.1,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting mahotas==0.6.1\n Downloading https...,60.0


In [7]:
#_uid = r'[0-9a-zA-Z]{12,128}'
_line_number = r'(at line[:]*\s*\d+)'
#_uuid = r'[a-f0-9]{8}-[a-f0-9]{4}-4[a-f0-9]{3}-[89aAbB][a-f0-9]{3}-[a-f0-9]{12}'
_url = r'(http[s]|root|srm|file)*:(//|/)(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
_filepath = "(/[a-zA-Z\./]*[\s]?)"
path_regex = re.compile(r'(\b\w+://)\S+(?=\s)')
file_regex = re.compile(r'(\b[f|F]ile( exists)?:?\s?)/\S+(?=\s)')
py_regex = re.compile(r'/?\b[-./_a-zA-Z0-9]+\.py\b')
long_regex = re.compile(r'[-./_a-zA-Z0-9]{25,}')

In [8]:
def remove_whitespaces(sentence):
    return " ".join(sentence.split())

def substitute_path(string):
    string = path_regex.sub(r'\1*', string)
    string = py_regex.sub(r'*.py', string)
    string = file_regex.sub(r'\1*', string)
    string = long_regex.sub(r'*', string)
    return string

def cleaner(log_messages):
    for idx, item in enumerate(log_messages):
        item = re.sub(_line_number, "at line *", item)
        #item = re.sub(_uid, "*", item)
        #item = re.sub(_uuid, "*", item)
        item = re.sub(_url, "*", item)
        item = re.sub("\d+", "*", item)
        item = substitute_path(item)
        log_messages[idx] = remove_whitespaces(item)
    return log_messages

In [9]:
solver_total_errors_df['clean-messages'] = solver_total_errors_df['message']
clean_msg = cleaner(solver_total_errors_df['clean-messages'])

2020-06-18 21:47:33,906 [20278] WARNING  py.warnings:99: [JupyterRequire] /home/skotak/.local/share/virtualenvs/solver-error-classifier-SslAmjTI/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




In [11]:
def tokenization(log_messages):
    tokenized = []
    for line in log_messages:
        tokenized.append(TreebankWordTokenizer().tokenize(line))
    stop = stopwords.words('english') + list(punctuation) + ["``", "''", '""']
    cleaned_tokens = []
    for row in tokenized:
        #cleaned_tokens.append([i for i in row if i.lower() not in stop and not i.lower().isnumeric()])

        cleaned_tokens.append(list(filter(None, [i.strip(punctuation) for i in row if i.lower() not in stop and not i.lower().isnumeric()])))
    return cleaned_tokens

In [12]:
solver_total_errors_df['clean-messages'] = tokenization(solver_total_errors_df['clean-messages'])

In [17]:
solver_total_errors_df['clean-messages'][0]

['Command',
 'exited',
 'non-zero',
 'status',
 'code',
 'ERROR',
 'Command',
 'errored',
 'exit',
 'status',
 'command',
 'c',
 'import',
 'sys',
 'setuptools',
 'tokenize',
 'sys.argv',
 'tmp/pip-install-_gczw**.py',
 'file',
 'tmp/pip-install-_gczw**.py',
 'f=getattr',
 'tokenize',
 'open',
 'open',
 'file',
 'code=f.read',
 'replace',
 'r\\n',
 'n',
 'f.close',
 'exec',
 'compile',
 'code',
 'file',
 'exec',
 'egg_info',
 'egg-base',
 'tmp/pip-install-_gczw*i/tryton/pip-egg-info',
 'cwd',
 'tmp/pip-install-_gczw*i/tryton',
 'Complete',
 'output',
 'lines',
 'Traceback',
 'recent',
 'call',
 'last',
 'File',
 'string',
 'line',
 'module',
 'File',
 'tmp/pip-install-_gczw**.py',
 'line',
 'print',
 'makensis.exe',
 'found',
 'installers',
 'created',
 'SyntaxError',
 'Missing',
 'parentheses',
 'call',
 'print',
 'mean',
 'print',
 'makensis.exe',
 'found',
 'installers',
 'created',
 'ERROR',
 'Command',
 'errored',
 'exit',
 'status',
 'python',
 'py',
 'egg_info',
 'Check',
 'logs

In [14]:
solver_total_errors_df.head(10)

,package_name,package_version,index_url,type,command,message,return_code,stderr,stdout,timeout,clean-messages
0,tryton,3.2.20,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting tryton==3.2.20\n Downloading https...,60.0,"[Command, exited, non-zero, status, code, ERRO..."
1,pyobjc-framework-contacts,4.0b1,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting pyobjc-framework-contacts==4.0b1\n ...,60.0,"[Command, exited, non-zero, status, code, ERRO..."
2,numpy,1.14.0,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting numpy==1.14.0\n Downloading https:...,60.0,"[Command, exited, non-zero, status, code, ERRO..."
3,hachoir-core,1.3,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting hachoir-core==1.3\n Downloading ht...,60.0,"[Command, exited, non-zero, status, code, ERRO..."
4,xattr,0.7.7,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting xattr==0.7.7\n Downloading https:/...,60.0,"[Command, exited, non-zero, status, code, ERRO..."
5,spacy,2.1.1.dev0,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting spacy==2.1.1.dev0\n Downloading ht...,60.0,"[Command, exited, non-zero, status, code, ERRO..."
6,fastavro,0.16.5,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting fastavro==0.16.5\n Downloading htt...,60.0,"[Command, exited, non-zero, status, code, ERRO..."
7,pyeclib,1.6.0,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting pyeclib==1.6.0\n Downloading https...,60.0,"[Command, exited, non-zero, status, code, ERRO..."
8,happybase,0.8,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting happybase==0.8\n Downloading https...,60.0,"[Command, exited, non-zero, status, code, ERRO..."
9,mahotas,0.6.1,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting mahotas==0.6.1\n Downloading https...,60.0,"[Command, exited, non-zero, status, code, ERRO..."


In [15]:
solver_total_errors_df.to_csv('solver-error-clean-data.csv', index=False)

In [16]:
with open("solver-errors-clean-logs.txt", "wb") as fp: 
    pickle.dump(solver_total_errors_df['clean-messages'], fp)